In [10]:
from PIL import Image
import numpy as np
import glob
import csv

In [11]:
# 빈 Numpy 배열이 있어야 함
chairData = np.zeros(60* 360*360, dtype=np.int32).reshape(60, 360, 360)
laptopData = np.zeros(60*360*360, dtype=np.int32).reshape(60, 360, 360)
pianoData = np.zeros(60*360*360, dtype=np.int32).reshape(60, 360, 360)

i = 0
for filename in sorted(glob.glob('data/cnn_processed/chair/*.jpg')):
  img = np.array(Image.open(filename), dtype=np.int32)
  chairData[i,:,:] = img
  i += 1

i = 0
for filename in sorted(glob.glob('data/cnn_processed/laptop/*.jpg')):
  img = np.array(Image.open(filename), dtype=np.int32)
  laptopData[i,:,:] = img
  i += 1

i = 0
for filename in sorted(glob.glob('data/cnn_processed/piano/*.jpg')):
  img = np.array(Image.open(filename), dtype=np.int32)
  pianoData[i,:,:] = img
  i += 1
print(pianoData.shape)

(60, 360, 360)


In [12]:
train = np.concatenate((chairData, laptopData, pianoData), axis=0)
train = train.reshape(train.shape[0], train.shape[1], train.shape[2], 1) / 255
target = np.concatenate((np.zeros(60), np.ones(60), np.full(60, 2)), axis=0)

In [13]:
from tensorflow import keras
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=0.2)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2)

print('X_train.shape:', X_train.shape)
print('X_valid.shape:', X_valid.shape)
print('X_test.shape:', X_test.shape)
print('y_train.shape:', y_train.shape)
print('y_valid.shape:', y_valid.shape)
print('y_test.shape:', y_test.shape)

X_train.shape: (115, 360, 360, 1)
X_valid.shape: (29, 360, 360, 1)
X_test.shape: (36, 360, 360, 1)
y_train.shape: (115,)
y_valid.shape: (29,)
y_test.shape: (36,)


# model 만들기

In [14]:
modelCharacters = []
idx = 0

# 처음 들어가는 filter 갯수 조절
for i in [32, 64]:
  # 은닉층 갯수 조절
  for j in range(2):
    for _ in range(j+1):
      for bool in [ True, False ]:
          # for k in [0.2, 0.25, 0.3]:
            model = keras.Sequential()
            model.add(keras.layers.Conv2D(i, (3, 3), padding='same', input_shape=(360, 360, 1)))

            if bool:
              model.add(keras.layers.Conv2D(i, (3, 3), padding='same'))
            else:
              model.add(keras.layers.Conv2D(i, (5, 5), padding='same'))

            model.add(keras.layers.MaxPooling2D((2, 2)))
            
            model.add(keras.layers.Dropout(0.25))

            model.add(keras.layers.Flatten())
            model.add(keras.layers.Dense(i, activation='relu'))
            model.add(keras.layers.Dense(3, activation='softmax'))


            # 모델 컴파일
            model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

            checkpoint_cb = keras.callbacks.ModelCheckpoint(
                'results/cnn1/res'+str(idx)+'.h5',
                monitor='val_loss',
                verbose=1,
                save_best_only=True,
                mode='min'
            )
            history = model.fit(
                X_train,
                y_train,
                epochs=20,
                validation_data=(X_valid, y_valid),
                callbacks=[checkpoint_cb]
            )
            # read best model
            model = keras.models.load_model('results/cnn1/res'+str(idx)+'.h5')

            # test 데이터로 평가
            test_loss, test_acc = model.evaluate(X_test, y_test)
            print('test_loss:', test_loss)
            print('test_acc:', test_acc)

            modelCharacter = {
              'idx': idx,
              'numberOfFilters': i,
              'numberOfLayers': j+1,
              'padding3': bool,
              'dropoutRatio': k,
              'test_loss': test_loss,
              'test_acc': test_acc
            }
            modelCharacters.append(modelCharacter)
            idx += 1

            with open('results/cnn_modelCharacters.csv', 'w', newline='') as f:
              writer = csv.writer(f)
              writer.writerow(['idx', 'numberOfFilters', 'numberOfLayers', 'padding3', 'dropoutRatio', 'test_loss', 'test_acc'])
              for modelCharacter in modelCharacters:
                writer.writerow([modelCharacter['idx'], modelCharacter['numberOfFilters'], modelCharacter['numberOfLayers'], modelCharacter['padding3'], modelCharacter['dropoutRatio'], modelCharacter['test_loss'], modelCharacter['test_acc']])
    

# save modelCharacters as csv




Epoch 1/20
4/4 [==============================] - ETA: 0s - loss: 20.5395 - accuracy: 0.3217
Epoch 1: val_loss improved from inf to 7.09277, saving model to results/cnn1/res0.h5
4/4 [==============================] - 23s 6s/step - loss: 20.5395 - accuracy: 0.3217 - val_loss: 7.0928 - val_accuracy: 0.5517
Epoch 2/20
4/4 [==============================] - ETA: 0s - loss: 5.0315 - accuracy: 0.5565
Epoch 2: val_loss improved from 7.09277 to 3.31649, saving model to results/cnn1/res0.h5
4/4 [==============================] - 23s 6s/step - loss: 5.0315 - accuracy: 0.5565 - val_loss: 3.3165 - val_accuracy: 0.6207
Epoch 3/20
4/4 [==============================] - ETA: 0s - loss: 7.4644 - accuracy: 0.4522
Epoch 3: val_loss improved from 3.31649 to 1.80341, saving model to results/cnn1/res0.h5
4/4 [==============================] - 20s 5s/step - loss: 7.4644 - accuracy: 0.4522 - val_loss: 1.8034 - val_accuracy: 0.6897
Epoch 4/20
4/4 [==============================] - ETA: 0s - loss: 1.7229 - acc

KeyboardInterrupt: 